# Initialization
---
This cell downloads and extracts the dataset from https://drive.google.com/file/d/1sJ97q9cdPhxlcnH1utNVql2nMUpRY_lt/view?usp=sharing.
- Execute it **ONLY ONCE**, at the start of your work.

In [1]:
import gdown
downloaded = gdown.download(
    "https://drive.google.com/file/d/1sJ97q9cdPhxlcnH1utNVql2nMUpRY_lt/view?usp=sharing",
    output="fever_test.filtered.sampled.csv",
    quiet=True,
    use_cookies=True,
    fuzzy=True
)

These cells initialize the models and the dataset.
- You need to execute it **ONLY ONCE**, but, if for any reason the process crashes, you may try re-running from this cell (so you'll avoid downloading files again).
- If it still crashes, then re-run from the start.

In [2]:
import random
import polars as pl
random.seed(3983751073717997123)
TRAIN_PATH = 'nli_fever/train_fitems.jsonl'
data_df = pl.read_csv("fever_test.filtered.sampled.csv", separator=',')
print(data_df.head())
print(data_df.height, 'samples')

shape: (5, 9)
┌─────┬────────┬──────────────┬──────────────┬───┬──────────────┬─────────────┬───────────┬────────┐
│ id  ┆ cid    ┆ premise      ┆ hypothesis   ┆ … ┆ label        ┆ new         ┆ new label ┆ change │
│ --- ┆ ---    ┆ ---          ┆ ---          ┆   ┆ ---          ┆ hypothesis  ┆ ---       ┆ type   │
│ i64 ┆ i64    ┆ str          ┆ str          ┆   ┆ str          ┆ ---         ┆ str       ┆ ---    │
│     ┆        ┆              ┆              ┆   ┆              ┆ str         ┆           ┆ str    │
╞═════╪════════╪══════════════╪══════════════╪═══╪══════════════╪═════════════╪═══════════╪════════╡
│ 0   ┆ 58846  ┆ Johnny       ┆ Johnny       ┆ … ┆ NEUTRAL      ┆             ┆           ┆        │
│     ┆        ┆ Galecki . He ┆ Galecki has  ┆   ┆              ┆             ┆           ┆        │
│     ┆        ┆ is known f…  ┆ been in at … ┆   ┆              ┆             ┆           ┆        │
│ 1   ┆ 172460 ┆ Matteo Renzi ┆ Matteo Renzi ┆ … ┆ ENTAILMENT   ┆            

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

MODELS = {}
TOKENIZERS = {}

model_name_base = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
model_name_large = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
model_name_large_2 = "Joelzhang/deberta-v3-large-snli_mnli_fever_anli_R1_R2_R3-nli"

for model_name in [model_name_base, model_name_large]:
    print(f"> loading {model_name}")
    TOKENIZERS[model_name] = AutoTokenizer.from_pretrained(model_name)
    MODELS[model_name] = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)


def inference(model_name, premise, hypothesis):
    model_input = TOKENIZERS[chosen_model](premise, hypothesis, truncation=False, return_tensors="pt")
    output = MODELS[chosen_model](model_input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    label_names = ["ENTAILMENT", "NEUTRAL", "CONTRADICTION"]
    return {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}

> loading MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/870M [00:00<?, ?B/s]

---
# The Main Loop
This cell contains the main part of the program: it will loop through each sample of the dataset, asking you to provide a new, hard to understand, hypothesis for each of them.

You can choose either to:
1. modify the given hypothesis, keeping the same label
2. come up with a new hypothesis, keeping the same label (you can also use ChatGPT for ideas)

In both cases, when writing the result on [this google sheet](https://docs.google.com/spreadsheets/d/1k7JTOOS2jUDItxCh7xSjwf3eGR8skGP7P7HQGh7_WCg/edit#gid=0), write also the main "change" you performed.
- You can come up with your categorization or take inspiration from the one of [this paper](https://arxiv.org/pdf/2010.12729) (see Table 2).

NOTE: **The changes on the hypothesis can be anything as long as the label does not change**.

NOTE 2: in case you go for option 2 and write a new starting hypothesis H", be sure to check that the models can actually predict the correct class for H" (which ideally should be the same label as the original hypothesis H). To do this, use this other [Colab]()

---
### Formal Definition
**Given**:
- *M* :   ensemble of models that you will fool
- *P* :   premise (the 'context')
- *H* :   hypothesis (the 'claim'), simple enough so that *M* correctly classifies the relationship between *P* and *H*
- *L* :   gold label (the relationship between *P* and *H*)

**Task**: generate *H'* such that:
1. *H* and *H'* have more or less the same meaning --> the relationship between *P* and *H'* is the same as the relationship between *P* and *H*
2. *H'* can fool *M* --> *M* will predict a different relationship type

In [ ]:
last = int(input("If you are resuming, enter the last ID you worked on (otherwise 0): "))
assert last < data_df.height, f"You entered an ID value that is higher than the size of the dataset -- Rerun this cell."

for i, elem in enumerate(data_df.iter_rows(named=True)):
    if i <= last:
        continue
    chosen_model = random.choice(list(MODELS.keys()))
    print("-"*30)
    print(f"[ID {i} - CID {elem['cid']} - model to fool: {chosen_model}]")
    print(f"PREMISE:")
    for context in elem['premise'].split('.'):
        if context.strip() != '':
            print(f"\t> {context.strip()}.")
    print(f"HYPOTHESIS:\n\t> {elem['hypothesis']}")
    print(f"GOLD LABEL: {elem['label']}")
    print("-"*30)

    hypothesis = input("> type new hypothesis: ")
    while hypothesis.lower() != 'n':
        prediction = inference(chosen_model, elem['premise'], hypothesis)
        predicted = max(prediction, key=prediction.get)
        if predicted != elem["label"]:
            print(f"PREDICTED LABEL **CHANGED**: >>>> {predicted} <<<< -- {prediction}", flush=True)
        else:
            print(f"PREDICTED LABEL: {predicted} -- {prediction}", flush=True)
        hypothesis = input("type n to exit, otherwise type new hyphotesis: ")

------------------------------
[ID 56 - CID 113664 - model to fool: MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli]
PREMISE:
	> Uganda.
	> Beginning in 1894 , the area was ruled as a protectorate by the British , who established administrative law across the territory.
HYPOTHESIS:
	> The British did not establish administrative law in Uganda.
GOLD LABEL: CONTRADICTION
------------------------------
